In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [2]:
import numpy as np
from tqdm import trange
from tqdm import tqdm
from docplex.mp.model import Model

import model
import model_parameters as MP

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [3]:
print(f"number of campaigns {MP.C}")
print(f"number of customers {MP.U}")
print(f"number of channels {MP.H}")
print(f"number of planning days {MP.D}")
print(f"number of quota categories {MP.I}")
print(f"number of priority categories {MP.P}")

number of campaigns 10
number of customers 1000
number of channels 3
number of planning days 7
number of quota categories 3
number of priority categories 10


#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
MP.e_cu

array([[0, 0, 0, ..., 0, 1, 1],
       [1, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 0, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 1, 1]])

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [5]:
MP.q_ic

array([[1, 0, 0, 1, 1, 1, 0, 0, 1, 1],
       [1, 0, 0, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 1, 0, 1, 1, 0]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [6]:
MP.rp_c

array([41, 17, 22, 17, 33, 67, 33, 41, 17, 41])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [7]:
MP.b

7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [8]:
MP.k

3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [9]:
MP.l_c

array([2, 3, 4, 3, 2, 2, 4, 3, 3, 3])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [10]:
(MP.m_i, MP.n_i)

(array([4, 3, 4]), array([1, 1, 2]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [11]:
MP.t_hd

array([[700., 700., 600., 500., 500., 700., 700.],
       [600., 500., 500., 600., 700., 700., 500.],
       [700., 500., 500., 500., 700., 500., 600.]])

### Model

In [12]:
mdl = Model(name='Campaign Optimization')

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [13]:
X_cuhd = {(c,u,h,d): mdl.binary_var(f"X_c:{c}_u:{u}_h:{h}_d:{d}")
            for c in range(0,MP.C)
            for u in range(0,MP.U) 
            for h in range(0,MP.H)
            for d in range(0,MP.D)}

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [14]:
maximize = mdl.maximize(mdl.sum([X_cuhd[(c,u,h,d)] * MP.rp_c[c]
                  for c in range(0,MP.C)
                  for u in range(0,MP.U) 
                  for h in range(0,MP.H) 
                  for d in range(0,MP.D)]))

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall c \in C,\forall u \in U,\forall h \in H,\forall d \in D
$$

In [15]:
eligibilitiy = mdl.add_constraints(
    (X_cuhd[(c,u,h,d)] <= MP.e_cu[c,u]
    for c in range(0,MP.C)
    for u in range(0,MP.U) 
    for h in range(0,MP.H) 
    for d in range(0,MP.D))
)

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [16]:
one_channel = mdl.add_constraints(
    ((mdl.sum(X_cuhd[(c,u,h,d)] for h in range(0,MP.H)) <= 1)
    for c in range(0,MP.C)
    for u in range(0,MP.U) 
    for d in range(0,MP.D))
)

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [17]:
weekly_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)] 
               for d in range(0,MP.D) 
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.b)
        for u in range(0,MP.U)))

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [18]:
daily_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.k)
        for d in range(0,MP.D)
        for u in range(0,MP.U)))

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [19]:
campaign_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for h in range(0,MP.H) 
               for d in range(0,MP.D)) <= MP.l_c[c] )
        for c in range(0,MP.C)
        for u in range(0,MP.U)))

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [20]:
weekly_quota = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*MP.q_ic[i,c]
               for c in range(0,MP.C)
               for h in range(0,MP.H) 
               for d in range(0,MP.D)) <= MP.m_i[i])
        for u in range(0,MP.U)
        for i in range(0,MP.I)))

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [21]:
daily_quota = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*MP.q_ic[i,c]
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.n_i[i])
        for u in range(0,MP.U)
        for d in range(0,MP.D)
        for i in range(0,MP.I)))

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [22]:
channel_capacity = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]
               for u in range(0,MP.U) 
               for c in range(0,MP.C)) <= MP.t_hd[h,d])
        for h in range(0,MP.H)
        for d in range(0,MP.D)))

### Solution

In [23]:
%%time

solution = mdl.solve(log_output=True, time_limit=600)
#solution = mdl.solve(log_output=True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
CPXPARAM_TimeLimit                               600
Found incumbent of value 0.000000 after 0.02 sec. (7.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 293504 rows and 116760 columns.
MIP Presolve modified 882 coefficients.
Reduced MIP has 28517 rows, 93240 columns, and 483210 nonzeros.
Reduced MIP has 93240 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.22 sec. (633.13 ticks)
Probing time = 0.22 sec. (23.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 58 rows and 42 columns.
Reduced MIP has 28459 rows, 93198 columns, and 481698 nonzeros.
Reduced MIP has 93198 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.40 sec. (480.29 ticks)
Probing time = 0.24 sec. (23.26 ticks)
Clique table members: 20272.
MIP emphasis: balance optimality and feasibil

In [24]:
solution.objective_value

262149.0

In [25]:
solution.as_df().sum()["value"]

6658.0

In [26]:
X_cuhd2 = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')
for ky,_ in solution.as_name_dict().items():
    exec(f'X_cuhd2{[int(i.split(":")[1]) for i in ky.split("_")[1:]]} = 1', {}, {'X_cuhd2':X_cuhd2})

c_i = 0
u_i = 1
h_i = 2
d_i = 3

mdl = model.Model([
    model.Constraint('eligibility',MP.eligibility, (c_i, u_i, h_i, d_i,)),
    model.Constraint('channel_capacity',MP.channel_capacity, (h_i, d_i,)),
    model.Constraint('daily_limitation',MP.daily_limitation, (u_i, d_i,)),
    model.Constraint('weekly_limitation',MP.weekly_limitation, (u_i,)),
    model.Constraint('campaign_limitation',MP.campaign_limitation, (c_i, u_i,)),
    model.Constraint('daily_quota',MP.daily_quota, (u_i, d_i,)),
    model.Constraint('one_channel',MP.one_channel, (c_i, u_i, d_i,)),
    model.Constraint('weekly_quota',MP.weekly_quota, (u_i,))
], MP.objective_fn)

def validate():
    for c in trange(MP.C, desc="Campaigns Loop"):
        for d in trange(MP.D, desc=f"Days Loop for campaign-{c}"):
            for h in range(MP.H):
                for u in range(MP.U):
                    if X_cuhd2[c,u,h,d]==1 and not mdl.execute(X_cuhd2, (c, u, h, d)):
                        raise RuntimeError(f'{(c, u, h, d)} does not consistent with previous values!')
    print("Solution is consistent with greedy from mip respect")
    
def anti_validate():
    for c in trange(MP.C, desc="Campaigns Loop"):
        for d in trange(MP.D, desc=f"Days Loop for campaign-{c}"):
            for h in range(MP.H):
                for u in range(MP.U):
                    if X_cuhd2[c,u,h,d]==0:
                        X_cuhd2[c,u,h,d]=1
                        if mdl.execute(X_cuhd2, (c, u, h, d)):
                            raise RuntimeError(f'{(c, u, h, d)} should failed')
                        else:
                            X_cuhd2[c,u,h,d]=0
    print("Solution is consistent with greedy from greedy respect")

validate()
anti_validate()

model_value = mdl.calc_value(X_cuhd2)
model_comm_count = X_cuhd2.sum()

if (solution.objective_value == model_value):
    print(f"Solution value ({model_value}) is consistent with greedy")
else:
    print(f"Solution value ({model_value}) is not consistent with greedy")
    
if (solution.as_df().sum()["value"] == model_comm_count):
    print(f"Solution communication count ({model_comm_count}) is consistent with greedy")
else:
    print(f"Solution communication count ({model_comm_count}) is consistent with greedy")

Days Loop for campaign-1: 100%|██████████| 7/7 [00:00<00:00, 79.27it/s]

Days Loop for campaign-3: 100%|██████████| 7/7 [00:00<00:00, 160.16it/s]

Days Loop for campaign-5: 100%|██████████| 7/7 [00:00<00:00, 30.38it/s]

Days Loop for campaign-8: 100%|██████████| 7/7 [00:00<00:00, 184.22it/s]

Days Loop for campaign-0:   0%|          | 0/7 [00:00<?, ?it/s]

Solution is consistent with greedy from mip respect



Campaigns Loop: 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]

Solution is consistent with greedy from greedy respect
Solution value (262149) is consistent with greedy
Solution communication count (6658) is consistent with greedy
